##Installs


In [ ]:
!pip install torch
!pip install transformers datasets
!pip install --upgrade accelerate

##Load Model and Tokenizers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")




##Load Dataset

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset

notebook_login()

##prompt and utterance

In [ ]:
xtrain = load_dataset("empathetic_dialogues", split = "train").shape
xval = load_dataset("empathetic_dialogues", split = "validation").shape
xtest = load_dataset("empathetic_dialogues", split = "test").shape

Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Dataset empathetic_dialogues downloaded and prepared to /root/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf. Subsequent calls will reuse this data.


In [ ]:
xtrain.num_rows



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'num_rows'

In [ ]:
#The original dataset is huge, shuffle then sample

train = load_dataset("empathetic_dialogues", split = "train[:3600]").shuffle(seed=420)
val = load_dataset("empathetic_dialogues", split = "validation[:1200]").shuffle(seed=420)
test = load_dataset("empathetic_dialogues", split = "test[:1200]").shuffle(seed=420)


Found cached dataset empathetic_dialogues (C:/Users/darre/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf)
Found cached dataset empathetic_dialogues (C:/Users/darre/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf)
Found cached dataset empathetic_dialogues (C:/Users/darre/.cache/huggingface/datasets/empathetic_dialogues/default/0.1.0/09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf)


NameError: ignored

In [ ]:
emp_dia_train = train.select_columns(['prompt', 'utterance'])
emp_dia_val = train.select_columns(['prompt',  'utterance'])
emp_dia_test = train.select_columns(['prompt',  'utterance'])

#emp_dia_train = emp_dia_train.rename_columns({'prompt': 'Context', 'context': 'Knowledge', 'utterance': 'Response' })
emp_dia_train = emp_dia_train.rename_columns({'prompt': 'Context',  'utterance': 'Response' })
emp_dia_test = emp_dia_test.rename_columns({'prompt': 'Context',  'utterance': 'Response' })
emp_dia_val = emp_dia_val.rename_columns({'prompt': 'Context',  'utterance': 'Response' })

##Pre-Processing

In [ ]:
emp_dia_train

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3600
})

In [ ]:
from transformers import DataCollatorForLanguageModeling ##does batching
from transformers import Seq2SeqTrainingArguments, TrainingArguments, Trainer, Seq2SeqTrainer
from torch.utils.data import DataLoader

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

def tokenize_function(example):
  return tokenizer(example['Context'], example['Response'], truncation = True,max_length = 32)

tokenized_train = emp_dia_train.map(tokenize_function, batched = True)
tokenized_test = emp_dia_test.map(tokenize_function, batched = True)
tokenized_val = emp_dia_val.map(tokenize_function, batched = True)

#outputs = model(input_ids, decoder_input_ids=train['input_ids'])

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\darre\.cache\huggingface\datasets\empathetic_dialogues\default\0.1.0\09bbeed3882a67db98c73952fb3c1c9a85af83dc78f81454c2454382fd03f6cf\cache-b786c705e75e08cd.arrow


In [ ]:
#Tokenizer Stuffs

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

##Training

In [ ]:

training_args = TrainingArguments(
    output_dir="DarrenLo/fine-tuned-dialogpt-pal",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    do_eval = True,
    num_train_epochs=2,
    weight_decay=0.1,
    learning_rate=5e-4,
    push_to_hub=True,
)
print('ok)')
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,

)

trainer.train()



ok)


C:\Users\darre\DarrenLo/fine-tuned-dialogpt-pal is already a clone of https://huggingface.co/DarrenLo/fine-tuned-dialogpt-pal. Make sure you pull the latest changes with `repo.git_pull()`.
C:\Anaconda\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=450, training_loss=2.011668158637153, metrics={'train_runtime': 2913.2287, 'train_samples_per_second': 2.471, 'train_steps_per_second': 0.154, 'total_flos': 417915312537600.0, 'train_loss': 2.011668158637153, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.35G [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.81k [00:00<?, ?B/s]

Upload file runs/May18_18-12-39_DESKTOP-2LG01LF/1684397562.7736266/events.out.tfevents.1684397562.DESKTOP-2LG0…

Upload file runs/May18_18-12-39_DESKTOP-2LG01LF/events.out.tfevents.1684397562.DESKTOP-2LG01LF.22540.0:   0%| …

To https://huggingface.co/DarrenLo/fine-tuned-dialogpt-pal
   f47574a..491ef8d  main -> main

To https://huggingface.co/DarrenLo/fine-tuned-dialogpt-pal
   491ef8d..232a566  main -> main



'https://huggingface.co/DarrenLo/fine-tuned-dialogpt-pal/commit/491ef8dcf428964536191958016cf592e0ef5fe4'

In [ ]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_new_tokens = 16, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

NameError: ignored

In [ ]:
user_input = input()

while user_input != 'exit':
  encode_input = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
  bot_input = model.generate(encode_input, max_length=32, pad_token_id=tokenizer.eos_token_id)
  print(tokenizer.decode(bot_input))

Hi


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


TypeError: ignored

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: ignored